In [1]:
import os 
import pandas as pd
    
import json
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import glob
import allel
import itertools
from matplotlib import gridspec
import matplotlib.pyplot as plt
import pickle
from statannot import add_stat_annotation
from scipy import stats
import matplotlib.cm as cm
import matplotlib

from math import pi
import scipy

import matplotlib.patches as mpatches
import statsmodels.stats as sts_m
import mne
import statsmodels.api as sm
from tqdm import tqdm

In [2]:
pd.options.display.max_columns=200
pd.options.display.max_rows=100
from pandas.core.common import SettingWithCopyWarning
import warnings
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


In [3]:
df_met = pd.read_csv("../results/data/processed_hmf_escape_info.tsv.gz",sep="\t")
df_primary = pd.read_csv("../results/data//processed_pcawg_escape_info.tsv.gz",sep="\t")
df_meta = pd.read_csv("../metadata/dataset_metadata_supp_table3.tsv",sep="\t")

/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (16,17,19,20,142) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
def perform_regression_control_tmb(zscore_query,zscore_control,ytrain):
    # defining the dependent and independent variables
    t=pd.DataFrame(np.array([zscore_query]).T,columns=["zscore_query"])
    t["zscore_control"] = zscore_control
    t["r"] = ytrain
    Xtrain = t[["zscore_query","zscore_control"]]
    ytrain =t[["r"]]
    # building the model and fitting the data
    Xtrain = sm.add_constant(Xtrain)
    log_reg = sm.Logit(ytrain, Xtrain,missing="drop").fit(disp=0)
    params = log_reg.params
    conf = log_reg.conf_int()
    conf['Odds Ratio'] = params
    conf.columns = ['5%', '95%', 'Odds Ratio']
    conf["pvalue"] = log_reg.pvalues
    return conf

def perform_regression_simple(zscore_query,ytrain):
    # defining the dependent and independent variables
    t=pd.DataFrame(np.array([zscore_query]).T,columns=["zscore_query"])
    t["r"] = ytrain
    Xtrain = t[["zscore_query"]]
    ytrain =t[["r"]]
    # building the model and fitting the data
    Xtrain = sm.add_constant(Xtrain)
    log_reg = sm.Logit(ytrain, Xtrain,missing="drop").fit(disp=0)
    params = log_reg.params
    conf = log_reg.conf_int()
    conf['Odds Ratio'] = params
    conf.columns = ['5%', '95%', 'Odds Ratio']
    conf["pvalue"] = log_reg.pvalues
    return conf


def ttype_regression(df,variables_query=["genetic_immune_escape"],columns_query=[]):
    ct = df_comb["cancer_type_code"].value_counts()
    ls=[]
    for ttype in list(ct[ct>=15].index):
        v=df[df["cancer_type_code"]==ttype]
        samples=v["sample_id"].values
        for col in columns_query:
            q=v[col].values
            mean_v = np.nanmean(q)
            std_v = np.nanstd(q)
            zscore_query = [(v - mean_v) / std_v for v in q]
            for variable_query in variables_query:
                Y=v[variable_query].values
                if sum(Y) < 5: # if there are less than 5 samples with the variable_reg == True, then do not perform regression, lack of informative results
                    continue
                try:
                    c=perform_regression_simple(zscore_query,Y)
                except: # failed to converge?
                    continue 
                c["column"] = col
                c["ttype"] = ttype
                c["mean_v"] = mean_v
                c["n_samples_exposure"] = len(samples)
                c["variable_dependent"] = variable_query
                ls.append(c)
    f=pd.concat(ls).sort_values("pvalue")
    f=f.loc["zscore_query"]
    l_corrected=[]
    for variable_query in variables_query: # perform a variable-specfic pvalue correction
        c=f[f["variable_dependent"]==variable_query]
        qvalues=mne.stats.fdr_correction(c["pvalue"])[1]
        c["qvalue"] = qvalues
        l_corrected.append(c)
    return pd.concat(l_corrected)
    
def ttype_regression_control(df,df_global,variables_query=["genetic_immune_escape"],type_control="age"):
    ls=[]
    ct = df_global["cancer_type_code"].value_counts() # obtain number of samples per cancer type
    thresholds={"SBS":100,"DBS":25,"ID":50} # threshold to consider the mut signatures
    for ttype in list(ct[ct>=15].index): # only those ttype with >= 15 samples
        ttype_info=df_global[df_global["cancer_type_code"]==ttype]
        samples=ttype_info["sample_id_2"].values
        if type_control == "age": # then use molecular age TMB
            q=df.loc[[("SBS","Age, 5mC deamination","SBS1"),("SBS","Age","SBS5")]].sum(axis=0)[samples].values
        else: # use global TMB as control
            q=df_global.set_index("sample_id_2").loc[samples]["smnv_load"]
        mean_control,std_control = np.nanmean(q), np.nanstd(q)
        if std_control == 0:
            print (ttype, "Only one sample in std.")
            continue # only one sample?
        zscores_control = [(v - mean_control) / std_control for v in q] # zscore the values of control
        for type_mut in ["SBS","DBS","ID"]: # for each type of mutation, obtain the mutational signatures with high prevalence and exposure
            threshold=thresholds[type_mut]
            counts=df[df>threshold].count(axis=1).sort_values(ascending=False)  
            for sbst in counts.loc[type_mut].index:
                if "Age" in sbst[0] or "Unknown" in sbst[0]:
                    continue # only mutational process with known aetiology
                sbs=tuple([type_mut]+list(sbst)) # prepare the index
                samples_exposure=sum([s>threshold for s in df.loc[sbs][samples]])
                if samples_exposure < 15: # less than 20 samples with threshold mutations, do not perform the analysis
                    continue
                # zscore the variable of mut. signature
                mean_v,std_v = np.nanmean(df.loc[sbs][samples]), np.nanstd(df.loc[sbs][samples])
                zscore_query = [(v - mean_v) / std_v for v in df.loc[sbs][samples]]
                for variable_query in variables_query:  # for each dependent variable, perform regression
                    Y=ttype_info[variable_query].values
                    if sum(Y) < 5: # regression not meaningful, less than 5 samples with value == True
                        continue
                    try:
                        c=perform_regression_control_tmb(zscore_query,zscores_control,Y)
                    except: # failed to converge
                        continue
                    c["column"] = "__".join(list(sbs))
                    c["ttype"] = ttype
                    c["mean_v"] = mean_v
                    c["n_samples_exposure"] = samples_exposure
                    c["variable_dependent"] = variable_query
                    ls.append(c)
    f=pd.concat(ls).sort_values("pvalue")
    f=f.loc["zscore_query"]
    l_corrected=[]
    for variable_query in variables_query: # perform a variable-specfic pvalue correction
        c=f[f["variable_dependent"]==variable_query]
        qvalues=mne.stats.fdr_correction(c["pvalue"])[1]
        c["qvalue"] = qvalues
        l_corrected.append(c)
    return pd.concat(l_corrected)

### Load dependent variables

In [5]:
df_d = pd.read_csv("../results/data/features_correlation/randomized_escape_for_features.tsv.gz",sep="\t")
df_d1 = pd.read_csv("../results/data/features_correlation/escape_for_features.tsv",sep="\t")

# Tumor Mutation Burden

### Load TMB counts

In [6]:
df_comb = pd.read_csv("../results/data/features_correlation/sample_specific_info_tmb_full_data.tsv",sep="\t")
columns_tmb = ["smnv_load","frameshift","missense","svTumorMutationalBurden","sbs_load.clonal","sbs_load.subclonal","dbs_load.clonal","dbs_load.subclonal","indel_load.clonal","indel_load.subclonal","total_fusions","total_neo","clonal_neo","subclonal_neo","fusion_neo","mut_neo"]
columns_tmb+=["clonal_tmb","subclonal_tmb","snvs","indels","dbs"]

### Perform regression

In [14]:
df_comb = df_comb.merge(df_d).merge(df_d1)
l=[]
# Run variables of GIE
tmb_reg=ttype_regression(df_comb,variables_query=['selected_genetic_immune_escape',"excluding_loh_hla"],columns_query=columns_tmb)
# Run background randomized GIE
tmb_reg_randomized=ttype_regression(df_comb,variables_query=list(df_d.columns.values[101:301]),columns_query=columns_tmb)

/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1799: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1799: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1799: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1799: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1799: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1799: RuntimeW

In [15]:
tmb_reg.to_csv("../results/data/results_feature_correlation/tmb_regression.tsv",sep="\t",index=False)
tmb_reg_randomized.to_csv("../results/data/results_feature_correlation/tmb_regression_control.tsv",sep="\t",index=False)

# Mutatoinal signatures exposure

In [16]:
df_mut_signatures = pd.read_csv("../results/data/features_correlation/mutational_signatures_exposure.tsv",sep="\t").set_index(["mut_type","etiology","sig_name"])

In [17]:
l,l1=[],[]
df_global = df_meta.merge(df_d).merge(df_d1)
df_global["sample_id_2"] = df_global.apply(lambda r: r["sample_id_2"] if r["cohort"] == "Hartwig" else r["sample_id"],axis=1)
mut_sig_reg=ttype_regression_control(df_mut_signatures,df_global,variables_query=['selected_genetic_immune_escape',"excluding_loh_hla"])
mut_sig_reg_randomized=ttype_regression_control(df_mut_signatures,df_global,variables_query=list(df_d.columns.values[101:301]))

/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1799: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1799: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1799: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1799: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1799: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1799: RuntimeW

In [18]:
mut_sig_reg.to_csv("../results/data/results_feature_correlation/mut_sig_regression.tsv",sep="\t",index=False)
mut_sig_reg_randomized.to_csv("../results/data/results_feature_correlation/mut_sig_regression_control.tsv",sep="\t",index=False)

# Immune infiltration

In [19]:
df_infiltration = pd.read_csv("../results/data/features_correlation/immune_infiltration_stats.tsv",sep="\t")
columns_infiltration=['nk', 'infiltration_davoli', 'cd4_davoli',
       'cd8_davoli', 't_cell_grasso',"ifn-gamma"]
df_infiltration=df_infiltration.merge(df_d).merge(df_d1)
immune_reg=ttype_regression(df_infiltration,variables_query=['selected_genetic_immune_escape','excluding_loh_hla'],columns_query=columns_infiltration)
immune_reg_randomized=ttype_regression(df_infiltration,variables_query=list(df_d.columns.values[101:301]),columns_query=columns_infiltration)

/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/ipykernel_launcher.py:43: RuntimeWarning: Mean of empty slice
/home/fran/.local/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in double_scalars
/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/ipykernel_launcher.py:43: RuntimeWarning: Mean of empty slice
/home/fran/.local/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/ipykernel_launcher.py:43: RuntimeWarning: Mean of empty slice
/home/fran/.local/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/fran/miniconda3/envs/glo

In [20]:
immune_reg[immune_reg["qvalue"]<0.05].sort_values("qvalue")

,5%,95%,Odds Ratio,pvalue,column,ttype,mean_v,n_samples_exposure,variable_dependent,qvalue
zscore_query,0.262989,0.698713,0.480851,0.000015,ifn-gamma,COREAD,1.062452,421,selected_genetic_immune_escape,0.001823
zscore_query,0.296687,0.854425,0.575556,0.000052,ifn-gamma,COREAD,1.062452,421,excluding_loh_hla,0.003765
zscore_query,0.178317,0.615847,0.397082,0.000374,t_cell_grasso,COREAD,1.318136,421,selected_genetic_immune_escape,0.022224
zscore_query,0.168595,0.611411,0.390003,0.000556,infiltration_davoli,COREAD,0.409635,421,selected_genetic_immune_escape,0.022224
zscore_query,0.193957,0.789778,0.491868,0.001212,infiltration_davoli,COREAD,0.409635,421,excluding_loh_hla,0.043636


In [21]:
immune_reg.to_csv("../results/data/results_feature_correlation/immune_infiltration_regression.tsv",sep="\t",index=False)
immune_reg_randomized.to_csv("../results/data/results_feature_correlation/immune_infiltration_regression_control.tsv",sep="\t",index=False)

# Germline diversity 

In [22]:
columns_diversity = ["sum_diversity_germline","avg_divergence_germline"]
df_data = pd.concat([df_primary[["sample_id","cancer_type_code"]+columns_diversity],df_met[["sample_id","cancer_type_code"]+columns_diversity]])
df_data=df_data.merge(df_d).merge(df_d1)
diversity_reg=ttype_regression(df_data,variables_query=['selected_genetic_immune_escape','excluding_loh_hla'],columns_query=columns_diversity)
diversity_reg_randomized=ttype_regression(df_data,variables_query=list(df_d.columns.values[101:301]),columns_query=columns_diversity)

In [23]:
diversity_reg.to_csv("../results/data/results_feature_correlation/germ_diversity_regression.tsv",sep="\t",index=False)
diversity_reg_randomized.to_csv("../results/data/results_feature_correlation/germ_diversity_regression_control.tsv",sep="\t",index=False)

# Age

In [26]:
df_age = pd.read_csv("../results/data/features_correlation/age_cohort_features.tsv",sep="\t")[["sample_id","age"]].merge(df_meta[df_meta["is_selected"]==True]).merge(df_d).merge(df_d1)
# Run variables of GIE
age_reg=ttype_regression(df_age,variables_query=['selected_genetic_immune_escape','excluding_loh_hla'],columns_query=["age"])
# Run background randomized GIE
age_reg_randomized=ttype_regression(df_age,variables_query=list(df_d.columns.values[101:301]),columns_query=["age"])

/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/ipykernel_launcher.py:43: RuntimeWarning: Mean of empty slice
/home/fran/.local/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/fran/miniconda3/envs/global/lib/python3.6/site-packages/ipykernel_launcher.py:43: RuntimeWarning: Mean of empty slice
/home/fran/.local/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [27]:
age_reg.to_csv("../results/data/results_feature_correlation/age_regression.tsv",sep="\t",index=False)
age_reg_randomized.to_csv("../results/data/results_feature_correlation/age_regression_control.tsv",sep="\t",index=False)

In [28]:
age_reg[age_reg["qvalue"]<0.05].sort_values("qvalue")

,5%,95%,Odds Ratio,pvalue,column,ttype,mean_v,n_samples_exposure,variable_dependent,qvalue
zscore_query,0.328604,1.458202,0.893403,0.001933,age,DLBCL,53.223214,112,excluding_loh_hla,0.038667


In [29]:
age_reg = pd.read_csv("../results/data/results_feature_correlation/age_regression.tsv",sep="\t")

In [9]:
age_reg[age_reg["variable_dependent"]=="selected_genetic_immune_escape"].sort_values("pvalue")[["column","ttype","5%","95%","Odds Ratio","pvalue","qvalue"]]

,column,ttype,5%,95%,Odds Ratio,pvalue,qvalue
0,age,DLBCL,0.104090,0.955329,0.529709,0.014716,0.367899
1,age,SARC,-0.005624,1.741218,0.867797,0.051494,0.413597
2,age,PRAD,-0.023446,0.518994,0.247774,0.073368,0.413597
3,age,CESC,-0.060618,1.150111,0.544746,0.077782,0.413597
4,age,PAAD,-0.038044,0.554267,0.258112,0.087601,0.413597
5,age,LIHC,-0.769148,0.076010,-0.346569,0.107961,0.413597
6,age,OV,-0.080763,0.591226,0.255231,0.136528,0.413597
7,age,SKCM,-0.065507,0.458213,0.196353,0.141654,0.413597
8,age,GBM,-1.217181,0.200227,-0.508477,0.159657,0.413597
9,age,STAD,-0.139892,0.817194,0.338651,0.165439,0.413597


In [27]:
!libreoffice ../metadata/immune_selected_genes.tsv


In [32]:
age_reg[age_reg["variable_dependent"]=="selected_genetic_immune_escape"].sort_values("pvalue")

,5%,95%,Odds Ratio,pvalue,column,ttype,mean_v,n_samples_exposure,variable_dependent,qvalue
0,0.104090,0.955329,0.529709,0.014716,age,DLBCL,53.223214,112,selected_genetic_immune_escape,0.367899
1,-0.005624,1.741218,0.867797,0.051494,age,SARC,49.454545,88,selected_genetic_immune_escape,0.413597
2,-0.023446,0.518994,0.247774,0.073368,age,PRAD,65.621154,520,selected_genetic_immune_escape,0.413597
3,-0.060618,1.150111,0.544746,0.077782,age,CESC,45.280000,50,selected_genetic_immune_escape,0.413597
4,-0.038044,0.554267,0.258112,0.087601,age,PAAD,64.877483,302,selected_genetic_immune_escape,0.413597
5,-0.769148,0.076010,-0.346569,0.107961,age,LIHC,62.693878,147,selected_genetic_immune_escape,0.413597
6,-0.080763,0.591226,0.255231,0.136528,age,OV,62.037234,188,selected_genetic_immune_escape,0.413597
7,-0.065507,0.458213,0.196353,0.141654,age,SKCM,60.606299,381,selected_genetic_immune_escape,0.413597
8,-1.217181,0.200227,-0.508477,0.159657,age,GBM,57.505155,97,selected_genetic_immune_escape,0.413597
9,-0.139892,0.817194,0.338651,0.165439,age,STAD,62.134831,89,selected_genetic_immune_escape,0.413597
